<a href="https://colab.research.google.com/github/quanndm/kltn/blob/dev/notebooks/tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Init - import library - download data

In [1]:
!rm -rf kltn
!rm -rf sample_data
!git clone -b dev https://github.com/quanndm/kltn.git

Cloning into 'kltn'...
remote: Enumerating objects: 1374, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 1374 (delta 105), reused 83 (delta 43), pack-reused 1212 (from 2)
Receiving objects: 100% (1374/1374), 13.65 MiB | 18.76 MiB/s, done.
Resolving deltas: 100% (893/893), done.


In [2]:
from kltn.init.install_dependencies import install_packages, load_config
install_packages("./kltn")
config = load_config("./kltn/parameters.yaml")

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive

from kltn import *
from kltn.datasets.get_datasets import get_datasets_lits
from kltn.utils.logger import init_logger
from kltn.utils.train_val_epochs import  trainer_stage2
from kltn.utils.metrics import  DiceMetric, TverskyLossWSigmoid, DiceLossWSigmoid
from kltn.models.model_factory import ModelFactory
from kltn.utils.visualize_results import visualize_results

In [4]:
drive.flush_and_unmount()
drive.mount("/content/drive")

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [5]:
LOGGER = init_logger(config["log_path"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from kltn.datasets.prepare_data import prepare_dataset_LiTS
prepare_dataset_LiTS(dir_name=config["source_folder_lits"])

0.00% complete (down: 0.0 kB/s up: 0.0 kB/s peers: 1) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (udp://tracker.opentrackr.org:1337/announce)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
1.49% complete (down: 19134.7 kB/s up: 552.9 kB/s peers: 10) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://academictorrents.com/announce.php)[1

## Model

In [7]:
weight_path = "/content/drive/MyDrive/KLTN/code/weight/best_metric_model_UNet3DWResNeXtCoT_liver.pth"
model_stage_1 = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=1, n_channels=16).to(device)
model_stage_1.load_state_dict(torch.load(weight_path, map_location=device))

<All keys matched successfully>

In [8]:
model = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=1, n_channels=16).to(device)
model.load_state_dict({
    k: v for k, v in model_stage_1.state_dict().items() if "out" not in k
}, strict=False)

_IncompatibleKeys(missing_keys=['out.conv.weight', 'out.conv.bias'], unexpected_keys=[])

## Load data

In [9]:
full_train_dataset, val_dataset = get_datasets_lits(source_folder=config["source_folder_lits"], seed=123, fold_number=2, mode="tumor",model_stage_1=model_stage_1)
print(len(full_train_dataset), len(val_dataset))

105 26


In [10]:
train_loader = torch.utils.data.DataLoader(full_train_dataset, batch_size=1, shuffle=True,
                                           num_workers=0, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                         pin_memory=True, num_workers=0)

## Training

In [ ]:
criterion = DiceLossWSigmoid().to(device)

dice_acc = DiceMetric(include_background=False, reduction='mean_batch', get_not_nans=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=float(config["learning_rate_train"]), weight_decay=float(config["weight_decay"]))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(config["max_epochs"]), eta_min = float(config["eta_min"]))

In [12]:
LOGGER.info("[TRAINER] Start TRAIN process...")

(
    val_acc_max,
    best_epoch,
    dices_tumor,
    loss_epochs,
    trains_epoch,
    ious_tumor,
    precisions_tumor,
    recalls_tumor,
    time_tmp
) = trainer_stage2(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_func=criterion,
    acc_func=dice_acc,
    scheduler=scheduler,
    batch_size=config["batch_size"],
    max_epochs = config["max_epochs"],
    start_epoch = config["start_epoch"],
    val_every=config["val_every"],
    path_save_model=config["path_save_model_state"],
    logger=LOGGER,
    save_model=True,
    post_fix="tumor"
)

[TRAINER] Start TRAIN process...
INFO:kltn.utils.logger:[TRAINER] Start TRAIN process...

==============================Training epoch 1==============================
INFO:kltn.utils.logger:
==============================Training epoch 1==============================


Epoch 1/100 1/105 loss: 0.7601 time 3.53s
Epoch 1/100 2/105 loss: 0.7371 time 1.70s
Epoch 1/100 3/105 loss: 0.7461 time 6.15s
Epoch 1/100 4/105 loss: 0.7564 time 1.82s
Epoch 1/100 5/105 loss: 0.7590 time 5.70s
Epoch 1/100 6/105 loss: 0.7642 time 6.77s
Epoch 1/100 7/105 loss: 0.7643 time 5.45s
Epoch 1/100 8/105 loss: 0.7645 time 1.82s
Epoch 1/100 9/105 loss: 0.7636 time 6.33s
Epoch 1/100 10/105 loss: 0.7621 time 2.22s
Epoch 1/100 11/105 loss: 0.7615 time 3.10s
Epoch 1/100 12/105 loss: 0.7584 time 4.90s
Epoch 1/100 13/105 loss: 0.7456 time 9.26s
Epoch 1/100 14/105 loss: 0.7469 time 4.68s
Epoch 1/100 15/105 loss: 0.7476 time 1.14s
Epoch 1/100 16/105 loss: 0.7490 time 6.12s
Epoch 1/100 17/105 loss: 0.7503 time 6.79s
Epoch 1/100 18/105 loss: 0.7506 time 4.59s
Epoch 1/100 19/105 loss: 0.7502 time 1.94s
Epoch 1/100 20/105 loss: 0.7499 time 5.54s
Epoch 1/100 21/105 loss: 0.7504 time 1.67s
Epoch 1/100 22/105 loss: 0.7502 time 4.32s
Epoch 1/100 23/105 loss: 0.7453 time 1.79s
Epoch 1/100 24/105 l

Final training epochs: 1/100 ---[loss: 0.7185] ---[time 451.95s]
INFO:kltn.utils.logger:Final training epochs: 1/100 ---[loss: 0.7185] ---[time 451.95s]

********************Epoch 1 Validation********************
INFO:kltn.utils.logger:
********************Epoch 1 Validation********************


Epoch 1/100 105/105 loss: 0.7185 time 6.98s


Val 1/100 1/26, Dice_Tumor: 0.000000, time 2.70s
INFO:kltn.utils.logger:Val 1/100 1/26, Dice_Tumor: 0.000000, time 2.70s
Val 1/100 2/26, Dice_Tumor: 0.000000, time 1.42s
INFO:kltn.utils.logger:Val 1/100 2/26, Dice_Tumor: 0.000000, time 1.42s
Val 1/100 3/26, Dice_Tumor: 0.000000, time 7.30s
INFO:kltn.utils.logger:Val 1/100 3/26, Dice_Tumor: 0.000000, time 7.30s
Val 1/100 4/26, Dice_Tumor: 0.000000, time 7.84s
INFO:kltn.utils.logger:Val 1/100 4/26, Dice_Tumor: 0.000000, time 7.84s
Val 1/100 5/26, Dice_Tumor: 0.000000, time 1.65s
INFO:kltn.utils.logger:Val 1/100 5/26, Dice_Tumor: 0.000000, time 1.65s
Val 1/100 6/26, Dice_Tumor: 0.002252, time 5.19s
INFO:kltn.utils.logger:Val 1/100 6/26, Dice_Tumor: 0.002252, time 5.19s
Val 1/100 7/26, Dice_Tumor: 0.000000, time 5.60s
INFO:kltn.utils.logger:Val 1/100 7/26, Dice_Tumor: 0.000000, time 5.60s
Val 1/100 8/26, Dice_Tumor: 0.000000, time 4.03s
INFO:kltn.utils.logger:Val 1/100 8/26, Dice_Tumor: 0.000000, time 4.03s
Val 1/100 9/26, Dice_Tumor: 0.00

...

Final training epochs: 32/100 ---[loss: 0.3353] ---[time 492.82s]
INFO:kltn.utils.logger:Final training epochs: 32/100 ---[loss: 0.3353] ---[time 492.82s]

********************Epoch 32 Validation********************
INFO:kltn.utils.logger:
********************Epoch 32 Validation********************


Epoch 32/100 105/105 loss: 0.3353 time 1.72s


Val 32/100 1/26, Dice_Tumor: 0.000000, time 1.70s
INFO:kltn.utils.logger:Val 32/100 1/26, Dice_Tumor: 0.000000, time 1.70s
Val 32/100 2/26, Dice_Tumor: 0.154488, time 1.42s
INFO:kltn.utils.logger:Val 32/100 2/26, Dice_Tumor: 0.154488, time 1.42s
Val 32/100 3/26, Dice_Tumor: 0.000000, time 7.32s
INFO:kltn.utils.logger:Val 32/100 3/26, Dice_Tumor: 0.000000, time 7.32s
Val 32/100 4/26, Dice_Tumor: 0.019941, time 8.85s
INFO:kltn.utils.logger:Val 32/100 4/26, Dice_Tumor: 0.019941, time 8.85s
Val 32/100 5/26, Dice_Tumor: 0.274900, time 1.34s
INFO:kltn.utils.logger:Val 32/100 5/26, Dice_Tumor: 0.274900, time 1.34s
Val 32/100 6/26, Dice_Tumor: 0.194756, time 5.60s
INFO:kltn.utils.logger:Val 32/100 6/26, Dice_Tumor: 0.194756, time 5.60s
Val 32/100 7/26, Dice_Tumor: 0.000000, time 6.63s
INFO:kltn.utils.logger:Val 32/100 7/26, Dice_Tumor: 0.000000, time 6.63s
Val 32/100 8/26, Dice_Tumor: 0.000000, time 4.18s
INFO:kltn.utils.logger:Val 32/100 8/26, Dice_Tumor: 0.000000, time 4.18s
Val 32/100 9/26,

Epoch 33/100 1/105 loss: 0.1888 time 6.24s
Epoch 33/100 2/105 loss: 0.2140 time 2.55s
Epoch 33/100 3/105 loss: 0.2226 time 1.95s
Epoch 33/100 4/105 loss: 0.3138 time 7.23s
Epoch 33/100 5/105 loss: 0.3911 time 7.41s
Epoch 33/100 6/105 loss: 0.3807 time 5.96s
Epoch 33/100 7/105 loss: 0.3494 time 9.20s
Epoch 33/100 8/105 loss: 0.3538 time 1.24s
Epoch 33/100 9/105 loss: 0.3655 time 1.16s
Epoch 33/100 10/105 loss: 0.3438 time 1.62s
Epoch 33/100 11/105 loss: 0.3254 time 2.33s
Epoch 33/100 12/105 loss: 0.3167 time 1.79s
Epoch 33/100 13/105 loss: 0.3108 time 1.94s
Epoch 33/100 14/105 loss: 0.3121 time 6.85s
Epoch 33/100 15/105 loss: 0.3123 time 2.52s
Epoch 33/100 16/105 loss: 0.3196 time 3.09s
Epoch 33/100 17/105 loss: 0.3377 time 4.86s
Epoch 33/100 18/105 loss: 0.3275 time 9.29s
Epoch 33/100 19/105 loss: 0.3233 time 2.35s
Epoch 33/100 20/105 loss: 0.3364 time 6.72s
Epoch 33/100 21/105 loss: 0.3319 time 3.85s
Epoch 33/100 22/105 loss: 0.3457 time 3.66s
Epoch 33/100 23/105 loss: 0.3366 time 6.5

KeyboardInterrupt: 

In [ ]:
import json
path_save_result = f"/content/drive/MyDrive/KLTN/code/result_model_{model.__class__.__name__}_tumor.json"

results = {
    "dice_tumor": [float(x) for x in np.array(dices_tumor, dtype=np.float32)],
    "loss": [float(x) for x in np.array(loss_epochs, dtype= np.float32)],
    "iou_tumor": [float(x) for x in np.array(ious_tumor, dtype=np.float32)],
    "precision_tumor": [float(x) for x in np.array(precisions_tumor, dtype=np.float32)],
    "recall_tumor": [float(x) for x in np.array(recalls_tumor, dtype=np.float32)],
    "best_epoch": best_epoch,
    "time_train": time_tmp
}
with open(path_save_result, "w") as f:
    json.dump(results, f, indent=4)